# Hector

### Otros modelos

---



**SVM**

In [ ]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/My Drive/Machine Learning/TP3"


[Errno 2] No such file or directory: '/content/gdrive/My Drive/Machine Learning/TP3'
/content


In [ ]:
!pwd

/content


In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve

from Lib import Evaluator
from Lib import BMI_imputer
from Lib import SmokingStatus_imputer
from Lib import graficar_distribuciones_Train_Data
from Lib import graficar_distribuciones_Dev_Data
from Lib import flatten
from sklearn.metrics import log_loss
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

ModuleNotFoundError: ignored

### Path to the data on Drive

In [ ]:
#path axel
# DATA_HOME = r"/content/gdrive/My Drive/UBA/machine_learning/tp1"
# path Noe 
# DATA_HOME = r"/content/gdrive/My Drive/tp1"
# path Hector

DATA_HOME = r"/content/gdrive/MyDrive/Machine Learning/TP1/"
df = pd.read_csv(
    DATA_HOME + "/healthcare-dataset-stroke-data.csv"
)

In [ ]:
df.head()

### Crear las pipelines de operaciones sobre los datos

In [ ]:
bmi = ['bmi']
numericas = ['age', 'hypertension', 'avg_glucose_level']
categoricas = ["gender", "ever_married","smoking_status","heart_disease"]

# pipelines para bmi y otras variables numericas
imputer_pipe = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'), 
)

scaler_pipe = make_pipeline(
     StandardScaler()
)
# pipelines para variables categórcias
one_hot_pipe = make_pipeline(
    OneHotEncoder()
)
# preprocessor  sin modelado de missing values
preprocessor = make_column_transformer(
    (imputer_pipe, bmi),
    (scaler_pipe, numericas), 
    (one_hot_pipe, categoricas))

# preprocessor con modelado de missing value 
preprocessor2 = make_pipeline(
    make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
    ), 
    IterativeImputer(random_state=42)
)

### Imputamos bmi con una regresión lineal para predecir los missing values

In [ ]:
pipe_bmi_lr = make_pipeline(
    BMI_imputer("bmi"),
    make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
    ) 
 ) 

### Imputamos SmokingStatus con un árbol de decisión para predecir los missing values

In [ ]:
pipe_ModelImputer = make_pipeline(
     BMI_imputer("bmi"), 
     SmokingStatus_imputer("smoking_status"),
     make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
      )
    ) 

### Visualizar Diagramas de los pipelines de preprocesamiento de datos 

In [ ]:
# visualizar diagrama 
from sklearn import set_config
set_config(display='diagram')
display(preprocessor) 

In [ ]:
set_config(display='diagram')
display(preprocessor2) 

### Establecer las pipelines con los modelos

In [ ]:
SVM_Uncalibrated_pipe = make_pipeline(
    preprocessor, # con simpleImputer
    svm.SVC(random_state = 420, kernel = 'rbf', gamma = 0.7, C = 1.0,
        class_weight= 'balanced')  
)

SVM2_Uncalibrated_pipe = make_pipeline(
    preprocessor2, # con IterativeImputer
    svm.SVC(random_state = 420, kernel = 'rbf', gamma = 0.7, C = 1.0,
        class_weight= 'balanced')
)

SVM3_Uncalibrated_pipe  = make_pipeline(
    pipe_bmi_lr, # BMI Nan imputado con regresion lineal
    svm.SVC(random_state = 420, kernel = 'rbf', gamma = 0.7, C = 1.0,
        class_weight= 'balanced')
)

SVM4_Uncalibrated_pipe  = make_pipeline(
    pipe_ModelImputer, # BMI Nan imputado con regresion lineal y SmokingStatus imputao con DTC
    svm.SVC(random_state = 420, kernel = 'rbf', gamma = 0.7, C = 1.0,
        class_weight= 'balanced')
)

### Comparar la efectividad de un modelo SVM descalibrado con uno calibrado

### SVM descalibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.fit_and_eval_pipe('UncalibratedSMV', SVM_Uncalibrated_pipe)

In [ ]:
SVM_Uncalibrated_pipe.fit(xtrain, ytrain)
probs = SVM_Uncalibrated_pipe.decision_function(xdev)
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

### SVM calibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.eval_pipe_calibrated('UncalibratedSMV', SVM_Uncalibrated_pipe)

In [ ]:

ev = Evaluator(xtrain, ytrain, xdev, ydev)
ev.make_calibration_curve('UncalibratedSMV', SVM_Uncalibrated_pipe)

### Distribuciones de la variable respuesta con el Train Set

In [ ]:
calibrated = CalibratedClassifierCV(SVM_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Train_Data(xtrain, ytrain, calibrated)

### Distribuciones de la variable respuesta con el Dev Set


In [ ]:
calibrated = CalibratedClassifierCV(SVM_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Dev_Data(xdev, ydev, calibrated)

### Comparar la efectividad de los modelos SVM entrenados y probados con datasets preprocesados de tres manera diferentes 

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)

ev.eval_pipe_calibrated('solo SVM con SimpleImputer', SVM_Uncalibrated_pipe)
ev.eval_pipe_calibrated('SVM con IterativeImputer de bmi', SVM2_Uncalibrated_pipe)
ev.eval_pipe_calibrated('SVM con regresion de bmi', SVM3_Uncalibrated_pipe)
ev.eval_pipe_calibrated('SVM con regresion de bmi y DTC en ss', SVM4_Uncalibrated_pipe)

pd.DataFrame(ev.evaluations)

### Selección de Hiperparámetros en modelos SVM

In [ ]:
from pprint import pprint
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from time import time


def objective(params):
    print(params)
    SVMUncalibrated_pipe = make_pipeline(
    preprocessor, 
    svm.SVC(random_state = 435, kernel = 'rbf', class_weight= 'balanced', **params)  
    )
    t0 = time()
    calibrated = CalibratedClassifierCV(SVMUncalibrated_pipe, method = 'isotonic', cv = 10)
    calibrated.fit(xtrain, ytrain)
    train_time = time() - t0
    trainloss = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1])
    devloss = log_loss(ydev, calibrated.predict_proba(xdev)[:,1])
    testloss = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
    print(f'trainloss {trainloss:.02f}', f'devloss {devloss:.02f}', f'testloss {testloss:.02f}')
    return dict(
        tr_loss = trainloss,
        loss = devloss,
        test_loss = testloss,
        params = params,
        train_time = train_time,
        status=STATUS_OK
    )


In [ ]:
space = {
    'gamma': hp.loguniform('gamma', low = np.log(0.01), high = np.log(3)),
    'C': hp.loguniform('C value', low = np.log(0.01), high = np.log(10)),
}

trials = Trials()

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

gamma = hp.loguniform('gamma', low = np.log(0.01), high = np.log(3))
samples = [hyperopt.pyll.stochastic.sample(gamma) for i in range(1000)]
plt.hist(samples)

In [ ]:
Cvalue = hp.loguniform('C value', low = np.log(0.01), high = np.log(10))
samples = [hyperopt.pyll.stochastic.sample(Cvalue) for i in range(1000)]
plt.hist(samples)

In [ ]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

### Mostrar los mejores modelos

In [ ]:
df_res_hyperopt = pd.DataFrame(list(map(flatten, [e['result'] for e in trials.trials])))
df_res_hyperopt.sort_values('loss').head()

### Guardar el dataframe (Descargarlo o cambiar el path para preservarlo)

In [ ]:
#df_res_hyperopt.to_pickle("df_res_hyperopt")

# Para cargar el archivo de vuelta: 
# df_res_hyperopt = pd.read_pickle("df_res_hyperopt") 

In [ ]:
# df_res_hyperopt = pd.read_pickle("/content/gdrive/MyDrive/Machine Learning/TP3/df_res_hyperopt_SVM") 

### Graficar los errores de Train (naranja) y Dev (azul) y mostrar el mejor modelo 

In [ ]:
df_res_hyperopt.loss.plot(style='-o', alpha=0.5)
plt.scatter([df_res_hyperopt.loss.argmin()], [df_res_hyperopt.loss.min()], c='r')
df_res_hyperopt.tr_loss.plot()
plt.yscale('log')
plt.grid()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_res_hyperopt.tr_loss, df_res_hyperopt.loss, c=(df_res_hyperopt.loss-df_res_hyperopt.tr_loss)/df_res_hyperopt.loss*100)
plt.title('Comparison of training and dev losses.\n Color corresponds to overfitting percentage')
plt.colorbar()
m = min(df_res_hyperopt.tr_loss.min(), df_res_hyperopt.loss.min())
M = max(df_res_hyperopt.tr_loss.max(), df_res_hyperopt.loss.max())
plt.plot([m, M], [m, M], 'k--')
plt.xlabel('tr loss')
plt.ylabel('dev loss')
plt.grid()

### Mostrar la distribución de los hiperparámetros para los mejores y peores modelos

In [ ]:
df = df_res_hyperopt
cut_point = df.loss.median()
best_models_df = df[df.loss <= cut_point]
worst_models_df = df[df.loss > cut_point]

def visualize_param(param_name):
  s = df[f'params.{param_name}']
  if s.dtype.name == 'object':
    visualize_categorical_param(param_name)
  else: # assume numerical
    visualize_numerical_param(param_name)

def visualize_categorical_param(param_name):
    pd.concat([
      best_models_df[f'params.{param_name}'].value_counts().rename('best'),
      worst_models_df[f'params.{param_name}'].value_counts().rename('worst')
  ], axis=1).plot.bar()

def visualize_numerical_param(param_name):
  plt.violinplot([
      best_models_df[f'params.{param_name}'],
      worst_models_df[f'params.{param_name}']
  ])
  plt.xticks([1, 2], ['best', 'worst'])

In [ ]:
param_names = list(trials.trials[0]['result']['params'].keys())
for param_name in param_names:
  plt.figure()
  visualize_param(param_name)
  plt.title(param_name)
  plt.tight_layout()

### Buscar el modelo que menos sobreajusta

In [ ]:
# El que menos overfittea de los mejors
best = df[df.loss < df.loss.min() * 1.001].sort_values('tr_loss', ascending = False).head(30)
best

### SVM  calibrado y optimizado

In [ ]:
params = {k.replace('params.', ''):v for k, v in best.iloc[0].to_dict().items() if 'params.' in k}
SVM_Uncalibrated_Pipe = make_pipeline(
    preprocessor, 
    svm.SVC(random_state = 435, kernel = 'rbf', class_weight= 'balanced', **params)  
)
t0 = time()
calibrated = CalibratedClassifierCV(SVM_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes = dict(
            name = "Mejor modelo SVM",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes

### Cambiando el "preprocesor" pipe por el "pipe_ModelImputer" pipe

In [ ]:
SVM_Uncalibrated_Pipe2 = make_pipeline(
    pipe_ModelImputer, 
    svm.SVC(random_state = 435, kernel = 'rbf', class_weight= 'balanced', **params)  
)
t0 = time()
calibrated = CalibratedClassifierCV(SVM_Uncalibrated_Pipe2, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes2 = dict(
            name = "Mejor modelo SVM",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes2

In [ ]:
train_time

### Evaluar el nivel de calibración de las probabilidades con un Diagrama de Confiabilidad

### Calibración con el dataset de Development

In [ ]:
calibrated = CalibratedClassifierCV(SVM_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
probs = calibrated.predict_proba(xdev)[:,1]
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle = '--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

### Calibración con el dataset de Testing

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.make_calibration_curve('CalibratedSMV', SVM_Uncalibrated_Pipe)

**RF**

In [ ]:
!pip install import_ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd "/content/gdrive/My Drive/Machine Learning/TP3"


In [ ]:
!pwd

In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
#from Lib import Evaluator

from Lib import BMI_imputer
from Lib import SmokingStatus_toNan
from Lib import SmokingStatus_imputer
from Lib import graficar_distribuciones_Train_Data
from Lib import graficar_distribuciones_Dev_Data
from Lib import flatten 
from sklearn.metrics import log_loss
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

### Path to the data on Drive

In [ ]:
DATA_HOME = r"/content/gdrive/MyDrive/Machine Learning/TP1/"
df = pd.read_csv(
    DATA_HOME + "/healthcare-dataset-stroke-data.csv"
)

In [ ]:
df.head()

### Limpieza de observaciones malas y particion x e y

In [ ]:
df = df[df.gender != "Other"]
df = df.drop(["work_type",	"Residence_type", "id"], axis=1)

In [ ]:
ydf = df["stroke"]
xdf = df.loc[:, df.columns != "stroke"]

### Division de dataset para validacion cruzada

In [ ]:
xtrain, xdev, ytrain, ydev = train_test_split(xdf, ydf, stratify=ydf, test_size=1/5, random_state=420)

In [ ]:
xdev, xtest, ydev, ytest = train_test_split(xdev, ydev, stratify=ydev, test_size=1/2, random_state=420)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from matplotlib import pyplot

class Evaluator:
    def __init__(self, X_train, y_train, X_dev, y_dev, X_test=None, y_test=None):
        self.X_train = X_train
        self.y_train = y_train
        self.X_dev = X_dev
        self.y_dev = y_dev
        self.X_test = X_test
        self.y_test = y_test
        self.evaluations = []

    def eval_pipe(self, model_name, pipe): 
        res = self.eval_prediction(model_name, pipe.predict_proba(self.X_train)[:,1], pipe.predict_proba(self.X_dev)[:,1])
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, pipe.predict_proba(self.X_test)[:,1])
        return res

    def fit_and_eval_pipe(self, model_name, pipe): 
        pipe.fit(self.X_train, self.y_train)
        res = self.eval_prediction(model_name, pipe.decision_function(self.X_train), pipe.decision_function(self.X_dev))
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, pipe.decision_function(self.X_test))
        return res

    def eval_pipe_calibrated(self, model_name, pipe): 
        #calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv = 10)
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv = 10)
        calibrated.fit(self.X_train, self.y_train)
        res = self.eval_prediction(model_name, calibrated.predict_proba(self.X_train)[:,1], calibrated.predict_proba(self.X_dev)[:,1])
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, calibrated.predict_proba(self.X_test)[:,1])
        return res

    def make_calibration_curve(self, model_name, pipe):
      if self.X_test is not None:
        # calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv=5)
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv=10)
        calibrated.fit(self.X_train, self.y_train)
        probs = calibrated.predict_proba(self.X_test)[:,1]
        # reliability diagram
        fop, mpv = calibration_curve(self.y_test, probs, n_bins = 10, normalize = True)
        # plot perfectly calibrated
        pyplot.plot([0, 1], [0, 1], linestyle='--')
        # plot model reliability
        pyplot.plot(mpv, fop, marker='.')
        pyplot.show()
      else:
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv=10)
        # calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv=5)
        calibrated.fit(self.X_train, self.y_train)
        probs = calibrated.predict_proba(self.X_dev)[:,1]
        # reliability diagram
        fop, mpv = calibration_curve(self.y_dev, probs, n_bins=10, normalize=True)
        # plot perfectly calibrated
        pyplot.plot([0, 1], [0, 1], linestyle='--')
        # plot model reliability
        pyplot.plot(mpv, fop, marker='.')
        pyplot.show()

    def eval_prediction(self, model_name, y_hat_train, y_hat_dev):
        res = dict(
            name=model_name,
            train= log_loss(self.y_train, y_hat_train),
            dev  = log_loss(self.y_dev, y_hat_dev, )
        )

        self.evaluations.append(res)
        return res

### Crear las pipelines de operaciones sobre los datos

In [ ]:
bmi = ['bmi']
numericas = ['age', 'hypertension','avg_glucose_level']
categoricas = ["gender", "ever_married","smoking_status","heart_disease"]

# pipes numericas
imputer_pipe = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'), 
)
scaler_pipe = make_pipeline(
     StandardScaler()
)
# pipes categoricas
one_hot_pipe = make_pipeline(
    OneHotEncoder()
)

# preprocessor 1  
preprocessor1 = make_column_transformer(
    (imputer_pipe, bmi),
    (scaler_pipe, numericas), 
    (one_hot_pipe, categoricas))

# preprocessor 2 

preprocessor2 = make_pipeline(
    SmokingStatus_toNan("smoking_status"),
    make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
    ), 
    IterativeImputer(random_state=42)
)

# preprocessor 3
preprocessor3 = make_pipeline(
     BMI_imputer("bmi"), 
     SmokingStatus_toNan("smoking_status"),
     SmokingStatus_imputer("smoking_status"),
     make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
      )
    )

### Visualizar Diagramas de los pipelines de preprocesamiento de datos 

In [ ]:
# visualizar diagrama 
from sklearn import set_config
set_config(display='diagram')
display(preprocessor1) 

In [ ]:
set_config(display='diagram')
display(preprocessor2) 

In [ ]:
set_config(display='diagram')
display(preprocessor3) 

### Establecer las pipelines con los modelos

In [ ]:
RF1_Uncalibrated_pipe = make_pipeline(
    preprocessor1, 
    RandomForestClassifier(random_state=420, class_weight = 'balanced')  
)

RF2_Uncalibrated_pipe = make_pipeline(
    preprocessor2, 
    RandomForestClassifier(random_state=420, class_weight = 'balanced') 
)

RF3_Uncalibrated_pipe  = make_pipeline(
    preprocessor3, 
    RandomForestClassifier(random_state=420, class_weight = 'balanced') 
)

### Comparar la efectividad de un modelo RF descalibrado con uno calibrado

RF descalibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
RF1_Uncalibrated_pipe.fit(xtrain, ytrain)
ev.eval_pipe('UncalibratedSMV', RF1_Uncalibrated_pipe)

In [ ]:
probs = RF1_Uncalibrated_pipe.predict_proba(xdev)[:,1]
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

RF calibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.eval_pipe_calibrated('UncalibratedSMV', RF1_Uncalibrated_pipe)

In [ ]:

ev = Evaluator(xtrain, ytrain, xdev, ydev)
ev.make_calibration_curve('UncalibratedSMV', RF1_Uncalibrated_pipe)

Distribuciones de la variable respuesta con el Train Set

In [ ]:
calibrated = CalibratedClassifierCV(RF1_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Train_Data(xtrain, ytrain, calibrated)

Distribuciones de la variable respuesta con el Dev Set


In [ ]:
calibrated = CalibratedClassifierCV(RF1_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Dev_Data(xdev, ydev, calibrated)

Comparar la efectividad de los modelos RF entrenados y probados con datasets preprocesados de tres manera diferentes 

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)

ev.eval_pipe_calibrated('solo RF con SimpleImputer', RF1_Uncalibrated_pipe)
ev.eval_pipe_calibrated('RF con IterativeImputer de bmi', RF2_Uncalibrated_pipe)
ev.eval_pipe_calibrated('RF con regresion de bmi y DTC en ss', RF3_Uncalibrated_pipe)

pd.DataFrame(ev.evaluations)

 Selección de Hiperparámetros en modelos RF

In [ ]:
from pprint import pprint
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from time import time


def objective(params):
    # Tenemos que hacer una adaptacion a nuestra funcion evaluate_model para que funcione con hyperopt
    if params['min_samples_split'] is not None:
        params['min_samples_split'] = int(params['min_samples_split'])
        params['max_features'] = int(params['max_features'])
        params['n_estimators'] = int(params['n_estimators'])
    print(params)
    RFUncalibrated_pipe = make_pipeline(
    preprocessor1, 
    RandomForestClassifier(random_state = 420, class_weight= 'balanced', **params)  
    )
    t0 = time()
    calibrated = CalibratedClassifierCV(RFUncalibrated_pipe, method = 'isotonic', cv = 10)
    calibrated.fit(xtrain, ytrain)
    train_time = time() - t0
    trainloss = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1])
    devloss = log_loss(ydev, calibrated.predict_proba(xdev)[:,1])
    testloss = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
    print(f'trainloss {trainloss:.02f}', f'devloss {devloss:.02f}', f'testloss {testloss:.02f}')
    return dict(
        tr_loss = trainloss,
        loss = devloss,
        test_loss = testloss,
        params = params,
        train_time = train_time,
        status=STATUS_OK
    )


In [ ]:
space = {
    'n_estimators': hp.loguniform('n_estimators', np.log(5), np.log(500)),
    'max_features': hp.uniform('max_features', 2, 8),
    'min_samples_split': hp.loguniform('min_samples_split', np.log(2), np.log(300))
}

trials = Trials()

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

n_estimators = hp.loguniform('gamma', low = np.log(0.01), high = np.log(3))
samples = [hyperopt.pyll.stochastic.sample(n_estimators) for i in range(1000)]
plt.hist(samples)

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

max_features = hp.uniform('max_features', 2, 8)
samples = [hyperopt.pyll.stochastic.sample(max_features) for i in range(1000)]
plt.hist(samples)

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

min_samples_split = hp.loguniform('min_samples_split', np.log(2), np.log(300))
samples = [hyperopt.pyll.stochastic.sample(min_samples_split) for i in range(1000)]
plt.hist(samples)

In [ ]:
best = fmin(objective, space, algo = tpe.suggest, max_evals = 100, trials = trials)

Mostrar los mejores modelos

In [ ]:
df_res_hyperopt = pd.DataFrame(list(map(flatten, [e['result'] for e in trials.trials])))
df_res_hyperopt.sort_values('loss').head()

 Guardar el dataframe (Descargarlo o cambiar el path para preservarlo)

In [ ]:
#df_res_hyperopt.to_pickle("df_res_hyperopt_RF")

# Para cargar el archivo de vuelta: 
# df_res_hyperopt = pd.read_pickle("df_res_hyperopt") 

In [ ]:
# df_res_hyperopt = pd.read_pickle("/content/gdrive/MyDrive/Machine Learning/TP3/df_res_hyperopt_SVM") 

Graficar los errores de Train (naranja) y Dev (azul) y mostrar el mejor modelo 

In [ ]:
df_res_hyperopt.loss.plot(style='-o', alpha=0.5)
plt.scatter([df_res_hyperopt.loss.argmin()], [df_res_hyperopt.loss.min()], c='r')
df_res_hyperopt.tr_loss.plot()
plt.yscale('log')
plt.grid()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_res_hyperopt.tr_loss, df_res_hyperopt.loss, c=(df_res_hyperopt.loss-df_res_hyperopt.tr_loss)/df_res_hyperopt.loss*100)
plt.title('Comparison of training and dev losses.\n Color corresponds to overfitting percentage')
plt.colorbar()
m = min(df_res_hyperopt.tr_loss.min(), df_res_hyperopt.loss.min())
M = max(df_res_hyperopt.tr_loss.max(), df_res_hyperopt.loss.max())
plt.plot([m, M], [m, M], 'k--')
plt.xlabel('tr loss')
plt.ylabel('dev loss')
plt.grid()

Mostrar la distribución de los hiperparámetros para los mejores y peores modelos

In [ ]:
df = df_res_hyperopt
cut_point = df.loss.median()
best_models_df = df[df.loss <= cut_point]
worst_models_df = df[df.loss > cut_point]

def visualize_param(param_name):
  s = df[f'params.{param_name}']
  if s.dtype.name == 'object':
    visualize_categorical_param(param_name)
  else: # assume numerical
    visualize_numerical_param(param_name)

def visualize_categorical_param(param_name):
    pd.concat([
      best_models_df[f'params.{param_name}'].value_counts().rename('best'),
      worst_models_df[f'params.{param_name}'].value_counts().rename('worst')
  ], axis=1).plot.bar()

def visualize_numerical_param(param_name):
  plt.violinplot([
      best_models_df[f'params.{param_name}'],
      worst_models_df[f'params.{param_name}']
  ])
  plt.xticks([1, 2], ['best', 'worst'])

In [ ]:
# param_names = list(trials.trials[0]['result']['params'].keys())
# for param_name in param_names:
#   plt.figure()
#   visualize_param(param_name)
#   plt.title(param_name)
#   plt.tight_layout()

Buscar el modelo que menos sobreajusta

In [ ]:
# El que menos overfittea de los mejors
best = df[df.loss < df.loss.min() * 1.001].sort_values('tr_loss', ascending = False).head(30)
best

 RF  calibrado y optimizado

In [ ]:
params = {k.replace('params.', ''):v for k, v in best.iloc[0].to_dict().items() if 'params.' in k}
# Tenemos que hacer una adaptacion a nuestra funcion evaluate_model para que funcione con hyperopt
if params['min_samples_split'] is not None:
    params['min_samples_split'] = int(params['min_samples_split'])
    params['max_features'] = int(params['max_features'])
    params['n_estimators'] = int(params['n_estimators'])
RF_Uncalibrated_Pipe = make_pipeline(
    preprocessor1, 
    RandomForestClassifier(random_state = 420, class_weight= 'balanced', **params) 
)
t0 = time()
calibrated = CalibratedClassifierCV(RF_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes = dict(
            name = "Mejor modelo RF",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes

Cambiando el "preprocesor" pipe por el "preprocessor3" pipe

In [ ]:
# Tenemos que hacer una adaptacion a nuestra funcion evaluate_model para que funcione con hyperopt
if params['min_samples_split'] is not None:
    params['min_samples_split'] = int(params['min_samples_split'])
    params['max_features'] = int(params['max_features'])
    params['n_estimators'] = int(params['n_estimators'])
RF_Uncalibrated_Pipe2 = make_pipeline(
    preprocessor3, 
    RandomForestClassifier(random_state = 420, class_weight= 'balanced', **params)  
)
t0 = time()
calibrated = CalibratedClassifierCV(RF_Uncalibrated_Pipe2, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes2 = dict(
            name = "Mejor modelo RF",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes2

In [ ]:
train_time

### Evaluar el nivel de calibración de las probabilidades con un Diagrama de Confiabilidad

### Calibración con el dataset de Development

In [ ]:
calibrated = CalibratedClassifierCV(RF_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
probs = calibrated.predict_proba(xdev)[:,1]
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle = '--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

 Calibración con el dataset de Testing

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.make_calibration_curve('CalibratedSMV', RF_Uncalibrated_Pipe)

### Con otros hiperparámetros se obtiene un rendimiento similar y al mismo 
tiempo los errores de train, dev y test son más parejos

In [ ]:
df_res_hyperopt = pd.DataFrame(list(map(flatten, [e['result'] for e in trials.trials])))
df_res_hyperopt.sort_values('loss').head()

In [ ]:
!pip install import_ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd "/content/gdrive/My Drive/Machine Learning/TP3"


In [ ]:
!pwd

In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
#from Lib import Evaluator
from Lib import BMI_imputer
from Lib import SmokingStatus_toNan
from Lib import SmokingStatus_imputer
from Lib import graficar_distribuciones_Train_Data
from Lib import graficar_distribuciones_Dev_Data
from Lib import flatten
from sklearn.metrics import log_loss
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

## Path to the data on Drive

In [ ]:
DATA_HOME = r"/content/gdrive/MyDrive/Machine Learning/TP1/"
df = pd.read_csv(
    DATA_HOME + "/healthcare-dataset-stroke-data.csv"
)

In [ ]:
df.head()

# Limpieza de observaciones malas y particion x e y

In [ ]:
df = df[df.gender != "Other"]
df = df.drop(["work_type",	"Residence_type", "id"], axis=1)

In [ ]:
ydf = df["stroke"]
xdf = df.loc[:, df.columns != "stroke"]

# Division de dataset para validacion cruzada

In [ ]:
xtrain, xdev, ytrain, ydev = train_test_split(xdf, ydf, stratify=ydf, test_size=1/5, random_state=420)

In [ ]:
xdev, xtest, ydev, ytest = train_test_split(xdev, ydev, stratify=ydev, test_size=1/2, random_state=420)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from matplotlib import pyplot

class Evaluator:
    def __init__(self, X_train, y_train, X_dev, y_dev, X_test=None, y_test=None):
        self.X_train = X_train
        self.y_train = y_train
        self.X_dev = X_dev
        self.y_dev = y_dev
        self.X_test = X_test
        self.y_test = y_test
        self.evaluations = []

    def eval_pipe(self, model_name, pipe): 
        res = self.eval_prediction(model_name, pipe.predict_proba(self.X_train)[:,1], pipe.predict_proba(self.X_dev)[:,1])
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, pipe.predict_proba(self.X_test)[:,1])
        return res

    def fit_and_eval_pipe(self, model_name, pipe): 
        pipe.fit(self.X_train, self.y_train)
        res = self.eval_prediction(model_name, pipe.decision_function(self.X_train), pipe.decision_function(self.X_dev))
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, pipe.decision_function(self.X_test))
        return res

    def eval_pipe_calibrated(self, model_name, pipe): 
        #calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv = 10)
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv = 10)
        calibrated.fit(self.X_train, self.y_train)
        res = self.eval_prediction(model_name, calibrated.predict_proba(self.X_train)[:,1], calibrated.predict_proba(self.X_dev)[:,1])
        if self.X_test is not None:
            res['test'] = log_loss(self.y_test, calibrated.predict_proba(self.X_test)[:,1])
        return res

    def make_calibration_curve(self, model_name, pipe):
      if self.X_test is not None:
        # calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv=5)
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv=10)
        calibrated.fit(self.X_train, self.y_train)
        probs = calibrated.predict_proba(self.X_test)[:,1]
        # reliability diagram
        fop, mpv = calibration_curve(self.y_test, probs, n_bins = 10, normalize = True)
        # plot perfectly calibrated
        pyplot.plot([0, 1], [0, 1], linestyle='--')
        # plot model reliability
        pyplot.plot(mpv, fop, marker='.')
        pyplot.show()
      else:
        calibrated = CalibratedClassifierCV(pipe, method = 'isotonic', cv=10)
        # calibrated = CalibratedClassifierCV(pipe, method = 'sigmoid', cv=5)
        calibrated.fit(self.X_train, self.y_train)
        probs = calibrated.predict_proba(self.X_dev)[:,1]
        # reliability diagram
        fop, mpv = calibration_curve(self.y_dev, probs, n_bins=10, normalize=True)
        # plot perfectly calibrated
        pyplot.plot([0, 1], [0, 1], linestyle='--')
        # plot model reliability
        pyplot.plot(mpv, fop, marker='.')
        pyplot.show()

    def eval_prediction(self, model_name, y_hat_train, y_hat_dev):
        res = dict(
            name=model_name,
            train= log_loss(self.y_train, y_hat_train),
            dev  = log_loss(self.y_dev, y_hat_dev, )
        )

        self.evaluations.append(res)
        return res

## Crear las pipelines de operaciones sobre los datos

In [ ]:
bmi = ['bmi']
numericas = ['age', 'hypertension','avg_glucose_level']
categoricas = ["gender", "ever_married","smoking_status","heart_disease"]

# pipes numericas
imputer_pipe = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'), 
)
scaler_pipe = make_pipeline(
     StandardScaler()
)
# pipes categoricas
one_hot_pipe = make_pipeline(
    OneHotEncoder()
)

# preprocessor   
preprocessor = make_column_transformer(
    (scaler_pipe, numericas),
    (imputer_pipe, bmi))

# preprocessor 1  
preprocessor1 = make_column_transformer(
    (imputer_pipe, bmi),
    (scaler_pipe, numericas), 
    (one_hot_pipe, categoricas))

# preprocessor 2 

preprocessor2 = make_pipeline(
    SmokingStatus_toNan("smoking_status"),
    make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
    ), 
    IterativeImputer(random_state=42)
)

# preprocessor 3
preprocessor3 = make_pipeline(
     BMI_imputer("bmi"), 
     SmokingStatus_toNan("smoking_status"),
     SmokingStatus_imputer("smoking_status"),
     make_column_transformer(
        (scaler_pipe, numericas), 
        (one_hot_pipe, categoricas)
      )
    )

## Visualizar Diagramas de los pipelines de preprocesamiento de datos 

In [ ]:
# visualizar diagrama 
from sklearn import set_config
set_config(display='diagram')
display(preprocessor1) 

In [ ]:
set_config(display='diagram')
display(preprocessor2) 

In [ ]:
set_config(display='diagram')
display(preprocessor3) 

## Establecer las pipelines con los modelos

In [ ]:
# from collections import Counter
# # print(Counter(ytrain))
# oversample = RandomOverSampler(sampling_strategy = 0.7)
# xover, yover = oversample.fit_resample(xtrain, ytrain)
# print(Counter(yover))
# #4087/199
# #4087 - 199
# #sum(ytrain)


In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# undersample = RandomUnderSampler(sampling_strategy = 'majority')
# xover, yover = undersample.fit_resample(xover, yover)
# print(Counter(yover))

In [ ]:
# Define GB pipeline:
over = RandomOverSampler(sampling_strategy = 0.7)
under = RandomUnderSampler(sampling_strategy = 'majority')
steps = [('o', over), ('u', under), ('model', GradientBoostingClassifier(random_state = 420, n_estimators = 20, learning_rate = 0.1,
                                 max_depth = 3))]
GB_pipe = Pipeline(steps=steps)

In [ ]:
GB_Uncalibrated_pipe = make_pipeline(
    preprocessor, 
    GB_pipe  
)

GB1_Uncalibrated_pipe = make_pipeline(
    preprocessor1, 
    GB_pipe  
)

GB2_Uncalibrated_pipe = make_pipeline(
    preprocessor2, 
    GB_pipe 
)

GB3_Uncalibrated_pipe  = make_pipeline(
    preprocessor3, 
    GB_pipe 
)

In [ ]:
# ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
# ev.eval_pipe_calibrated('UncalibratedSMV', GB1_Uncalibrated_pipe)

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)

ev.eval_pipe_calibrated('solo GB', GB_Uncalibrated_pipe)

## Comparar la efectividad de un modelo GB descalibrado con uno calibrado

## GB descalibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
GB1_Uncalibrated_pipe.fit(xtrain, ytrain)
ev.eval_pipe('UncalibratedSMV', GB1_Uncalibrated_pipe)

In [ ]:
probs = GB1_Uncalibrated_pipe.predict_proba(xdev)[:,1]
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

## GB calibrado

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.eval_pipe_calibrated('UncalibratedSMV', GB1_Uncalibrated_pipe)

In [ ]:

ev = Evaluator(xtrain, ytrain, xdev, ydev)
ev.make_calibration_curve('UncalibratedSMV', GB1_Uncalibrated_pipe)

## Distribuciones de la variable respuesta con el Train Set

In [ ]:
calibrated = CalibratedClassifierCV(GB1_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Train_Data(xtrain, ytrain, calibrated)

## Distribuciones de la variable respuesta con el Dev Set


In [ ]:
calibrated = CalibratedClassifierCV(GB1_Uncalibrated_pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
graficar_distribuciones_Dev_Data(xdev, ydev, calibrated)

## Comparar la efectividad de los modelos GB entrenados y probados con datasets preprocesados de tres manera diferentes 

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)

ev.eval_pipe_calibrated('solo GB', GB_Uncalibrated_pipe)
ev.eval_pipe_calibrated('GB con SimpleImputer', GB1_Uncalibrated_pipe)
ev.eval_pipe_calibrated('GB con IterativeImputer de bmi', GB2_Uncalibrated_pipe)
ev.eval_pipe_calibrated('GB con regresion de bmi y DTC en ss', GB3_Uncalibrated_pipe)

pd.DataFrame(ev.evaluations)

## Selección de Hiperparámetros en modelos GB

In [ ]:
from pprint import pprint
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from time import time


def objective(params):
    # Tenemos que hacer una adaptacion a nuestra funcion evaluate_model para que funcione con hyperopt
    params['n_estimators'] = int(params['n_estimators'])
    print(params)
    steps = [('o', over), ('u', under), ('model', GradientBoostingClassifier(random_state = 420, **params))]
    GB_pipe = Pipeline(steps = steps)
    GBUncalibrated_pipe = make_pipeline(
    preprocessor1, 
    GB_pipe  
    )
    t0 = time()
    calibrated = CalibratedClassifierCV(GBUncalibrated_pipe, method = 'isotonic', cv = 10)
    calibrated.fit(xtrain, ytrain)
    train_time = time() - t0
    trainloss = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1])
    devloss = log_loss(ydev, calibrated.predict_proba(xdev)[:,1])
    testloss = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
    print(f'trainloss {trainloss:.02f}', f'devloss {devloss:.02f}', f'testloss {testloss:.02f}')
    return dict(
        tr_loss = trainloss,
        loss = devloss,
        test_loss = testloss,
        params = params,
        train_time = train_time,
        status=STATUS_OK
    )


In [ ]:
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 300, 10),
    'max_depth': hp.uniform('max_depth', 2, 8),
    'subsample': hp.uniform('subsample', 0.2, 1.0),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1))
}

trials = Trials()

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

n_estimators = hp.loguniform('gamma', low = np.log(0.01), high = np.log(3))
samples = [hyperopt.pyll.stochastic.sample(n_estimators) for i in range(1000)]
plt.hist(samples)

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

max_depth = hp.uniform('max_depth', 2, 8)
samples = [hyperopt.pyll.stochastic.sample(max_depth) for i in range(1000)]
plt.hist(samples)

In [ ]:
import numpy as np
import hyperopt.pyll.stochastic

learning_rate = hp.loguniform('learning_rate', np.log(0.01), np.log(1))
samples = [hyperopt.pyll.stochastic.sample(learning_rate) for i in range(1000)]
plt.hist(samples)

In [ ]:
best = fmin(objective, space, algo = tpe.suggest, max_evals = 100, trials = trials)

## Mostrar los mejores modelos

In [ ]:
df_res_hyperopt = pd.DataFrame(list(map(flatten, [e['result'] for e in trials.trials])))
df_res_hyperopt.sort_values('loss').head()

## Guardar el dataframe (Descargarlo o cambiar el path para preservarlo)

In [ ]:
df_res_hyperopt.to_pickle("df_res_hyperopt_GB")

# Para cargar el archivo de vuelta: 
# df_res_hyperopt = pd.read_pickle("df_res_hyperopt") 

In [ ]:
# df_res_hyperopt = pd.read_pickle("/content/gdrive/MyDrive/Machine Learning/TP3/df_res_hyperopt_SVM") 

## Graficar los errores de Train (naranja) y Dev (azul) y mostrar el mejor modelo 

In [ ]:
df_res_hyperopt.loss.plot(style='-o', alpha=0.5)
plt.scatter([df_res_hyperopt.loss.argmin()], [df_res_hyperopt.loss.min()], c='r')
df_res_hyperopt.tr_loss.plot()
plt.yscale('log')
plt.grid()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_res_hyperopt.tr_loss, df_res_hyperopt.loss, c=(df_res_hyperopt.loss-df_res_hyperopt.tr_loss)/df_res_hyperopt.loss*100)
plt.title('Comparison of training and dev losses.\n Color corresponds to overfitting percentage')
plt.colorbar()
m = min(df_res_hyperopt.tr_loss.min(), df_res_hyperopt.loss.min())
M = max(df_res_hyperopt.tr_loss.max(), df_res_hyperopt.loss.max())
plt.plot([m, M], [m, M], 'k--')
plt.xlabel('tr loss')
plt.ylabel('dev loss')
plt.grid()

## Mostrar la distribución de los hiperparámetros para los mejores y peores modelos

In [ ]:
df = df_res_hyperopt
cut_point = df.loss.median()
best_models_df = df[df.loss <= cut_point]
worst_models_df = df[df.loss > cut_point]

def visualize_param(param_name):
  s = df[f'params.{param_name}']
  if s.dtype.name == 'object':
    visualize_categorical_param(param_name)
  else: # assume numerical
    visualize_numerical_param(param_name)

def visualize_categorical_param(param_name):
    pd.concat([
      best_models_df[f'params.{param_name}'].value_counts().rename('best'),
      worst_models_df[f'params.{param_name}'].value_counts().rename('worst')
  ], axis=1).plot.bar()

def visualize_numerical_param(param_name):
  plt.violinplot([
      best_models_df[f'params.{param_name}'],
      worst_models_df[f'params.{param_name}']
  ])
  plt.xticks([1, 2], ['best', 'worst'])

In [ ]:
param_names = list(trials.trials[0]['result']['params'].keys())
for param_name in param_names:
  plt.figure()
  visualize_param(param_name)
  plt.title(param_name)
  plt.tight_layout()

## Buscar el modelo que menos sobreajusta

In [ ]:
# El que menos overfittea de los mejors
best = df[df.loss < df.loss.min() * 1.001].sort_values('tr_loss', ascending = False).head(30)
best

## GB  calibrado y optimizado

In [ ]:
params = {k.replace('params.', ''):v for k, v in best.iloc[0].to_dict().items() if 'params.' in k}
params['n_estimators'] = int(params['n_estimators'])
print(params)
steps = [('o', over), ('u', under), ('model', GradientBoostingClassifier(random_state = 420, **params))]
GB_pipe = Pipeline(steps = steps)
GB_Uncalibrated_Pipe = make_pipeline(
preprocessor1, 
GB_pipe  
)
t0 = time()
calibrated = CalibratedClassifierCV(GB_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes = dict(
            name = "Mejor modelo GB",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes

## Cambiando el "preprocesor" pipe por el "preprocessor3" pipe

In [ ]:
# Tenemos que hacer una adaptacion a nuestra funcion evaluate_model para que funcione con hyperopt
params['n_estimators'] = int(params['n_estimators'])
print(params)
steps = [('o', over), ('u', under), ('model', GradientBoostingClassifier(random_state = 420, **params))]
GB_pipe = Pipeline(steps = steps)
GB_Uncalibrated_Pipe2 = make_pipeline(
preprocessor3, 
GB_pipe  
)
t0 = time()
calibrated = CalibratedClassifierCV(GB_Uncalibrated_Pipe2, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
train_time = time() - t0
BestModelRes2 = dict(
            name = "Mejor modelo GB",
            train_time = train_time,
            train = log_loss(ytrain, calibrated.predict_proba(xtrain)[:,1]),
            dev  = log_loss(ydev, calibrated.predict_proba(xdev)[:,1]),
            test = log_loss(ytest, calibrated.predict_proba(xtest)[:,1])
        )
BestModelRes2

In [ ]:
train_time

## Evaluar el nivel de calibración de las probabilidades con un Diagrama de Confiabilidad

## Calibración con el dataset de Development

In [ ]:
calibrated = CalibratedClassifierCV(GB_Uncalibrated_Pipe, method = 'isotonic', cv = 10)
calibrated.fit(xtrain, ytrain)
probs = calibrated.predict_proba(xdev)[:,1]
# reliability diagram
fop, mpv = calibration_curve(ydev, probs, n_bins = 10, normalize = True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle = '--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

## Calibración con el dataset de Testing

In [ ]:
ev = Evaluator(xtrain, ytrain, xdev, ydev, xtest, ytest)
ev.make_calibration_curve('CalibratedSMV', GB_Uncalibrated_Pipe)